In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import entropy
from sklearn.neighbors import KernelDensity

def analyze_phi_distributions(signal_phi1, signal_phi2, background_phi1, background_phi2, 
                            bins=50, bandwidth=0.1):
    """
    Analyze and visualize phi distributions for signal and background events.
    
    Parameters:
    -----------
    signal_phi1, signal_phi2: array-like
        Phi values for the two objects in signal events
    background_phi1, background_phi2: array-like
        Phi values for the two objects in background events
    bins: int
        Number of bins for histogram
    bandwidth: float
        Bandwidth parameter for kernel density estimation
    """
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(15, 10))
    
    # 2D scatter plot
    ax1 = plt.subplot(221)
    ax1.scatter(signal_phi1, signal_phi2, alpha=0.5, label='Signal', s=10)
    ax1.scatter(background_phi1, background_phi2, alpha=0.5, label='Background', s=10)
    ax1.set_xlabel('Φ₁')
    ax1.set_ylabel('Φ₂')
    ax1.set_title('2D Distribution of Φ Values')
    ax1.legend()
    
    # Compute KDE for both distributions
    X_sig = np.vstack([signal_phi1, signal_phi2]).T
    X_bkg = np.vstack([background_phi1, background_phi2]).T
    
    # Fit KDE
    kde_sig = KernelDensity(bandwidth=bandwidth).fit(X_sig)
    kde_bkg = KernelDensity(bandwidth=bandwidth).fit(X_bkg)
    
    # Create grid for density estimation
    x_grid = np.linspace(min(min(signal_phi1), min(background_phi1)),
                        max(max(signal_phi1), max(background_phi1)), 100)
    y_grid = np.linspace(min(min(signal_phi2), min(background_phi2)),
                        max(max(signal_phi2), max(background_phi2)), 100)
    X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
    xy_grid = np.vstack([X_grid.ravel(), Y_grid.ravel()]).T
    
    # Compute log densities
    Z_sig = np.exp(kde_sig.score_samples(xy_grid)).reshape(X_grid.shape)
    Z_bkg = np.exp(kde_bkg.score_samples(xy_grid)).reshape(X_grid.shape)
    
    # Plot signal KDE
    ax2 = plt.subplot(222)
    c2 = ax2.contourf(X_grid, Y_grid, Z_sig, levels=20, cmap='viridis')
    plt.colorbar(c2, ax=ax2)
    ax2.set_title('Signal Density Estimation')
    ax2.set_xlabel('Φ₁')
    ax2.set_ylabel('Φ₂')
    
    # Plot background KDE
    ax3 = plt.subplot(223)
    c3 = ax3.contourf(X_grid, Y_grid, Z_bkg, levels=20, cmap='viridis')
    plt.colorbar(c3, ax=ax3)
    ax3.set_title('Background Density Estimation')
    ax3.set_xlabel('Φ₁')
    ax3.set_ylabel('Φ₂')
    
    # Calculate KL divergence
    # Normalize distributions
    Z_sig_norm = Z_sig / Z_sig.sum()
    Z_bkg_norm = Z_bkg / Z_bkg.sum()
    
    # Add small constant to avoid division by zero
    epsilon = 1e-10
    Z_sig_norm = Z_sig_norm + epsilon
    Z_bkg_norm = Z_bkg_norm + epsilon
    
    # Calculate KL divergence in both directions
    kl_div_sig_bkg = entropy(Z_sig_norm.flatten(), Z_bkg_norm.flatten())
    kl_div_bkg_sig = entropy(Z_bkg_norm.flatten(), Z_sig_norm.flatten())
    
    # Plot difference
    ax4 = plt.subplot(224)
    diff = Z_sig_norm - Z_bkg_norm
    c4 = ax4.contourf(X_grid, Y_grid, diff, levels=20, cmap='RdBu')
    plt.colorbar(c4, ax=ax4)
    ax4.set_title('Density Difference (Signal - Background)')
    ax4.set_xlabel('Φ₁')
    ax4.set_ylabel('Φ₂')
    
    plt.tight_layout()
    
    return {
        'kl_divergence_sig_bkg': kl_div_sig_bkg,
        'kl_divergence_bkg_sig': kl_div_bkg_sig,
        'signal_kde': kde_sig,
        'background_kde': kde_bkg
    }